In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta

import pandas as pd
pd.set_option('display.max_rows', 200)
import plotly.express as px

from tqdm import tqdm

from data_handlers.runtimes import *

In [2]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [4]
# experiment_ids = ["dtn7ng", "dtn7rs", "ibrdtn", "serval"]

In [3]:
orig_event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/35
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/42
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/48


Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/70
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/94
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/26
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/51
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/87
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/69
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/63
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/14
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/7
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/19
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/80
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/13
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/64
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/4/56
Parsing /home/sterz/maci-docker-compose/maci_data

In [4]:
event_frame = orig_event_frame.copy()

In [5]:
event_frame = event_frame.replace("IBRDTN", "IBR-DTN")
event_frame = event_frame.replace("DTN7NG", "DTN7-ng")
event_frame = event_frame.replace("DTN7Rs", "DTN7-rs")


event_frame = event_frame.replace("mtcp", "MTCP")
event_frame = event_frame.replace("http", "HTTP")
event_frame = event_frame.replace("tcp", "TCPCL")
event_frame = event_frame.replace("quicl", "QUICL")

event_frame = event_frame.rename(columns={"Payload Size": "Bundle Size"})
event_frame = event_frame.rename(columns={"# Payloads": "# Bundles"})

event_frame["Software/CLP"] = event_frame['Software'] + "/" + event_frame['CLA']

event_frame = event_frame.replace("Serval/default", "Serval")

event_frame["Bundle Size"] = event_frame["Bundle Size"].replace(1000, "1 kB")
event_frame["Bundle Size"] = event_frame["Bundle Size"].replace(100000, "100 kB")
event_frame["Bundle Size"] = event_frame["Bundle Size"].replace(1000000, "1 MB")

In [6]:
creation_instances = event_frame[event_frame.Event == "creation"].groupby("Simulation ID")
delivery_instances = event_frame[event_frame.Event == "delivery"]

In [7]:
delivered_list = []
runtime_list = []
for sim_instance_id, creations in tqdm(creation_instances):
    deliveries_sid = delivery_instances[delivery_instances["Simulation ID"] == sim_instance_id]
    
    for _, row in creations.iterrows():
        # Compute deliverys
        bundle = row.Bundle
        deliveries = deliveries_sid[deliveries_sid.Bundle == bundle]

        was_delivered = not deliveries.empty

        row_dict = {
            "Simulation ID": sim_instance_id,
            "Bundle": bundle,
            "Delivered": was_delivered,
            "CLA": row.CLA,
            "Software/CLP": row["Software/CLP"],
            "Loss": row.Loss,
            "Bundle Size": row["Bundle Size"],
            "Software": row.Software,
            "# Nodes": row["# Nodes"],
            "# Bundles": row["# Bundles"],
        }
        delivered_list.append(row_dict)        
        
        # Compute runtimes
        if was_delivered:
            delivery_time = deliveries["Timestamp"].min()
            creation_time = row["Timestamp"]
            runtime = delivery_time - creation_time
            
            runtime_dict = {
                "Simulation ID": sim_instance_id,
                "Bundle": bundle,
                "Runtime": (runtime.to_pytimedelta() / timedelta(milliseconds=1)) / 1000,
                "CLA": row.CLA,
                "Software/CLP": row["Software/CLP"],
                "Loss": row.Loss,
                "Software": row.Software,
                "# Nodes": row["# Nodes"],
                "Bundle Size": row["Bundle Size"],
                "# Bundles": row["# Bundles"],
            }
            runtime_list.append(runtime_dict)
        
delivered_df = pd.DataFrame(delivered_list)
runtime_df = pd.DataFrame(runtime_list)

  0%|          | 0/96 [00:00<?, ?it/s]

100%|██████████| 96/96 [00:11<00:00,  8.43it/s]


In [8]:
row_dicts = []

for sim_instance_id, bundles in delivered_df.groupby("Simulation ID"):
    cla = bundles["CLA"].values[0]
    software_clp = bundles["Software/CLP"].values[0]
    loss = bundles["Loss"].values[0]
    software = bundles["Software"].values[0]
    nodes = bundles["# Nodes"].values[0]
    payload_size = bundles["Bundle Size"].values[0]
    payloads = bundles["# Bundles"].values[0]
    
    total_bundles =  payloads
    delivered_bundles = len(bundles[bundles.Delivered == True].index)
    
    row_dict = {
        "Simulation ID": sim_instance_id,
        "Delivered (%)": (float(delivered_bundles) / float(total_bundles)) * 100,
        "CLA": cla,
        "Software/CLP": software_clp,
        "Loss": loss,
        "Software": software,
        "# Nodes": nodes,
        "Bundle Size": payload_size,
        "# Bundles": payloads,
    }
    
    row_dicts.append(row_dict)

ratio_df = pd.DataFrame.from_dict(row_dicts)

In [15]:
fig = px.box(
    runtime_df[runtime_df["# Nodes"] == 2],
    x="Software/CLP", 
    y="Runtime",
    color="Loss",
    facet_col="# Bundles",
    facet_row="Bundle Size",
    labels={
        "Runtime": "",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles"
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLP": ["Serval", "IBR-DTN/TCPCL", "DTN7-rs/MTCP", "DTN7-rs/TCPCL", "DTN7-rs/HTTP", "DTN7-ng/MTCP", "DTN7-ng/QUICL"],
        "# Bundles": [10, 100]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Bundle Size: ", "")))

fig.add_annotation(x=-0.05, y=0.35,
                   text="Runtime (s)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
)

fig.write_image("runtime_2.pdf", width=700, height=300)
fig

In [16]:
fig = px.box(
    runtime_df[runtime_df["# Nodes"] == 32],
    x="Software/CLP", 
    y="Runtime",
    color="Loss",
    facet_col="# Bundles",
    facet_row="Bundle Size",
    labels={
        "Runtime": "",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLP": ["Serval", "IBR-DTN/TCPCL", "DTN7-rs/MTCP", "DTN7-rs/TCPCL", "DTN7-rs/HTTP", "DTN7-ng/MTCP", "DTN7-ng/QUICL"],
        "# Bundles": [10, 100]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Bundle Size: ", "")))

fig.add_annotation(x=-0.05, y=0.35,
                   text="Runtime (s)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
)

fig.write_image("runtime_32.pdf", width=700, height=300)
fig

In [17]:
fig = px.box(
    ratio_df[ratio_df["# Nodes"] == 2],
    x="Software/CLP", 
    y="Delivered (%)",
    color="Loss",
    facet_col="# Bundles",
    facet_row="Bundle Size",
    labels={
        "Runtime": "Runtime (s)",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
        "Delivered (%)": ""
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLP": ["Serval", "IBR-DTN/TCPCL", "DTN7-rs/MTCP", "DTN7-rs/TCPCL", "DTN7-rs/HTTP", "DTN7-ng/MTCP", "DTN7-ng/QUICL"],
        "# Bundles": [10, 100]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Bundle Size: ", "")))

fig.add_annotation(x=-0.04, y=0.35,
                   text="Successful Receptions (%)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
)

fig.write_image("delivered_2.pdf", width=700, height=300)
fig

In [18]:
fig = px.box(
    ratio_df[ratio_df["# Nodes"] == 32],
    x="Software/CLP", 
    y="Delivered (%)",
    color="Loss",
    facet_col="# Bundles",
    facet_row="Bundle Size",
    labels={
        "Runtime": "Runtime (s)",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
        "Delivered (%)": ""
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLP": ["Serval", "IBR-DTN/TCPCL", "DTN7-rs/MTCP", "DTN7-rs/TCPCL", "DTN7-rs/HTTP", "DTN7-ng/MTCP", "DTN7-ng/QUICL"],
        "# Bundles": [10, 100]
    },
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Bundle Size: ", "")))

fig.add_annotation(x=-0.04, y=0.35,
                   text="Successful Receptions (%)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
)

fig.write_image("delivered_32.pdf", width=700, height=300)
fig